# Calculate the correlations for each layer and create input RDMs

## Set up the environment

In [ ]:
import numpy as np
import os
import json
import matplotlib.pyplot as plt
import seaborn
import pandas as pd
import scipy 

## Select the model and layer

In [ ]:
NR_OF_SAMPLES = 60
num_layers = 7
num_timepoints = 8
layer_names = []
for layer_idx in range(num_layers):
    for timepoint in range(num_timepoints):       
        layer_time = 'ReLU_Layer_{}_Time_{}'.format(layer_idx,timepoint)
        layer_names.append(layer_time)

model_name = '/model_02.11_2/'
path_dnn_activ = '/scratch/agnek95/PDM/DATA/'

## Define the correlation function (correlation distance (Pearson) or Euclidean)

In [ ]:
def correlationd_matrix_pearson():
    file_name = os.path.join(path_dnn_activ+'RNN_ACTIVATIONS/activations/','{}_activations.npy'.format(layer_name)) 
    act = np.load(file_name,mmap_mode='r')
    print(np.argwhere(np.isnan(act)))
    print(act.shape)
    correlationd = 1-np.corrcoef(act)                                
    return(correlationd)

In [ ]:
def correlationd_matrix_euclid(): 
    file_name = os.path.join(path_dnn_activ+'RNN_ACTIVATIONS/activations/','{}_activations.npy'.format(layer_name)) 
    act = np.load(file_name,mmap_mode='r')
    print(np.argwhere(np.isnan(act)))
    print(act.shape)
    correlationd = np.zeros((NR_OF_SAMPLES,NR_OF_SAMPLES))
    correlationd[:] = np.nan
    
    for i in range(NR_OF_SAMPLES):
        for j in range(NR_OF_SAMPLES):  
            correlationd[i,j] = scipy.spatial.distance.euclidean(act[i,:],act[j,:])
            # print(correlationd[i,j])                                 
    return(correlationd)

# Create RDM

In [ ]:
corr_type='pearson'

for layer_name in layer_names:
    print('Calculating the correlations for model: ',model_name,'and layer: ',layer_name)
    if corr_type=='pearson':
        corr_matrix = correlationd_matrix_pearson()
    elif corr_type=='euclid':
        corr_matrix = correlationd_matrix_euclid()
    save_corr_path = os.path.join('/home/agnek95/SMST/PDM_FULL_EXPERIMENT/RESULTS_AVG/02.11_2_rnn/Input_RDM_{}'.format(corr_type), '{}_Input_RDM_{}.npy'.format(layer_name,corr_type))
    print("Save Input RDM -> {}".format(save_corr_path))
    np.save(save_corr_path, np.array(corr_matrix)) 


# To plot/visualize RDM

In [ ]:
filename_rdm='/scratch/agnek95/PDM/DATA/RNN_ACTIVATIONS/activations/ReLU_Layer_0_Time_0_activations.npy'
rdm_7_8 = np.load(filename_rdm,mmap_mode='r')
fig = plt.figure(figsize=(17,13))
ax = seaborn.heatmap(rdm_7_8, cmap='rainbow')

## In case the load function does not work

In [ ]:
# #If the load function gives an error, do this
# np_load_old = np.load # modify the default parameters of np.load
# np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
# activations_shape = np.load(path)
# np.load = np_load_old